In [1]:
import pyedflib
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow as tf
import scipy, pylab
import csv
index = ["1","2","3","4","5","6","7","8","9","10", 
         "11", "12", "13", "14","15","16","17","18","19","20"]

In [2]:
def read_stage(index,index_num):
    f = open("DatabaseSubjects/HypnogramR&K_subject"+index[index_num]+".txt")
    return [line[:-1] for line in f]

In [3]:
def read_edf(index,index_num):
    f= pyedflib.EdfReader("DatabaseSubjects/subject"+index[index_num]+".edf")
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((n, f.getNSamples()[0]))
    for i in np.arange(n):
        sigbufs[i, :]=f.readSignal(i)
    label = [ f.getLabel(1) ,  f.getLabel(2) ,  f.getLabel(15) ,  f.getLabel(16)
             ,  f.getLabel(17) ,  f.getLabel(18)]
    return label, sigbufs[1],  sigbufs[2], sigbufs[15], sigbufs[16], sigbufs[17], sigbufs[18],f

In [4]:
def stft(x, fs, framesz, hop):
    """
     x - signal
     fs - sample rate
     framesz - frame size
     hop - hop size (frame size = overlap + hop size)
    """
    framesamp = int(framesz*fs)
    hopsamp = int(hop*fs)
    w = np.hanning(framesamp)
    X = np.array([np.fft.rfft(w*x[i:i+framesamp]) 
                     for i in range(0, len(x)-framesamp, hopsamp)])
    return X

In [5]:
def padding(stft_):
    y_ = np.array(stft_.T)
    temp = []
    for i in range(0,len(y_)):
        temp.append(np.insert(y_[i],[0,-1],[y_[i][0],y_[i][-1]]))
    temp = np.reshape(temp,[len(y_),-1])    
        #temp = np.reshape(np.append(temp, np.insert(y_[i],[0,-1],[y_[i][0],y_[i][-1]])),[len(y_),-1]) 
    return temp

In [6]:
def split_frequency(padded_stft_):
    delta = np.array([padded_stft_[c] for c in range(1,5)])   
    theta = np.array([padded_stft_[c] for c in range(5,9)])
    alpha = np.array([padded_stft_[c] for c in range(9,13)])
    beta  = np.array([padded_stft_[c] for c in range(13,31)])   
    return delta, theta, alpha, beta

In [7]:
def split_time(signal_data): 
    temp= []
    for i in range(0, len(signal_data)):     
        temp.append(np.reshape(signal_data[i],[-1,10]))
    return np.array(temp)

In [8]:
def extract_feature(s_signal_data):
    mean = np.mean(s_signal_data,axis=2)
    mean = np.mean(mean,axis=0)
    mqv = mean_quadratic_value(s_signal_data)
    min_ = np.amin(s_signal_data,axis=2)
    min_ = np.amin(min_,axis=0)
    max_ = np.amax(s_signal_data,axis=2)
    max_ = np.amax(max_,axis=0)
    return (np.abs(mean), np.angle(mean), mqv,  np.abs(min_), np.abs(max_))

In [9]:
def mean_quadratic_value(s_signal_data):
    mqv = []
    scale = len(s_signal_data)*len(s_signal_data[0][0])
    for k in range(0, len(s_signal_data[0])):
        t_mqv = 0
        for  i in range(0, len(s_signal_data)):
            for j in range(0, len(s_signal_data[0][0])):
                t_mqv = t_mqv + s_signal_data[i][k][j]*s_signal_data[i][k][j]
        t_mqv = t_mqv/scale
        t_mqv = math.sqrt(np.abs(t_mqv))
        mqv.append(t_mqv)
    return mqv

In [10]:
def ratio_of_values(mqv_delta, mqv_theta, mqv_alpha, mqv_beta):
    delta = mqv_delta/(mqv_theta+mqv_alpha+mqv_beta)
    theta = mqv_theta/(mqv_delta+mqv_alpha+mqv_beta)
    alpha = mqv_alpha/(mqv_delta+mqv_theta+mqv_beta)
    beta  = mqv_beta/(mqv_delta+mqv_theta+mqv_alpha)
    return delta, theta, alpha, beta

In [11]:
def zip_feature(feature_data_0,feature_data_1,feature_data_2,
                feature_data_3,feature_data_4,feature_data_5):
    return np.array(feature_data_0+feature_data_1+feature_data_2+
                feature_data_3+feature_data_4+feature_data_5)

In [ ]:
fs = 200       # sampled at fs Hz
framesz = 1 # with a frame size of framesz  seconds
hop = 0.5    # and hop size of hop seconds.

In [ ]:
for c in range(1,len(index)):
    index_num = c
    stage = read_stage(index, index_num)
    #5sec r&k stage
    label, eeg0, eeg1, eeg2, eeg3, eeg4, eeg5,f = read_edf(index, index_num)
    f._close()
    #200HZ, eeg 0 : FP1 -A2 / eeg1 : CZ-A1 / eeg2 : O1-A2 /eeg3 FP2 - A1 / eeg4: O2-A1/ eeg5: CZ2-A1
    sleep_stage = np.array(stage[1:],int)
    
    # STFT.
    X0 = stft(eeg0, fs, framesz, hop)
    X1 = stft(eeg1, fs, framesz, hop)
    
    X2 = stft(eeg2, fs, framesz, hop)
    X3 = stft(eeg3, fs, framesz, hop)
    X4 = stft(eeg4, fs, framesz, hop)
    X5 = stft(eeg5, fs, framesz, hop)
    
    a,b,c,d,e,f = padding(X0),padding(X1),padding(X2),padding(X3),padding(X4),padding(X5)
    
    delta_0, theta_0, alpha_0, beta_0 = split_frequency(a)
    delta_1, theta_1, alpha_1, beta_1 = split_frequency(b)
    delta_2, theta_2, alpha_2, beta_2 = split_frequency(c)
    delta_3, theta_3, alpha_3, beta_3 = split_frequency(d)
    delta_4, theta_4, alpha_4, beta_4 = split_frequency(e)
    delta_5, theta_5, alpha_5, beta_5 = split_frequency(f)
    
    s_delta_0, s_theta_0, s_alpha_0, s_beta_0 = split_time(delta_0),split_time(theta_0),split_time(alpha_0),split_time(beta_0)
    s_delta_1, s_theta_1, s_alpha_1, s_beta_1 = split_time(delta_1),split_time(theta_1),split_time(alpha_1),split_time(beta_1)
    s_delta_2, s_theta_2, s_alpha_2, s_beta_2 = split_time(delta_2),split_time(theta_2),split_time(alpha_2),split_time(beta_2)
    s_delta_3, s_theta_3, s_alpha_3, s_beta_3 = split_time(delta_3),split_time(theta_3),split_time(alpha_3),split_time(beta_3)
    s_delta_4, s_theta_4, s_alpha_4, s_beta_4 = split_time(delta_4),split_time(theta_4),split_time(alpha_4),split_time(beta_4)
    s_delta_5, s_theta_5, s_alpha_5, s_beta_5 = split_time(delta_5),split_time(theta_5),split_time(alpha_5),split_time(beta_5)
    
    feature_delta_0 = extract_feature(s_delta_0)
    feature_delta_1 = extract_feature(s_delta_1)
    feature_delta_2 = extract_feature(s_delta_2)
    feature_delta_3 = extract_feature(s_delta_3)
    feature_delta_4 = extract_feature(s_delta_4)
    feature_delta_5 = extract_feature(s_delta_5)
    
    feature_theta_0 = extract_feature(s_theta_0)
    feature_theta_1 = extract_feature(s_theta_1)
    feature_theta_2 = extract_feature(s_theta_2)
    feature_theta_3 = extract_feature(s_theta_3)
    feature_theta_4 = extract_feature(s_theta_4)
    feature_theta_5 = extract_feature(s_theta_5)
    
    feature_alpha_0 = extract_feature(s_alpha_0)
    feature_alpha_1 = extract_feature(s_alpha_1)
    feature_alpha_2 = extract_feature(s_alpha_2)
    feature_alpha_3 = extract_feature(s_alpha_3)
    feature_alpha_4 = extract_feature(s_alpha_4)
    feature_alpha_5 = extract_feature(s_alpha_5)
    
    feature_beta_0 = extract_feature(s_beta_0)
    feature_beta_1 = extract_feature(s_beta_1)
    feature_beta_2 = extract_feature(s_beta_2)
    feature_beta_3 = extract_feature(s_beta_3)
    feature_beta_4 = extract_feature(s_beta_4)
    feature_beta_5 = extract_feature(s_beta_5)
    
    ratio0 = ratio_of_values(feature_delta_0[3],feature_theta_0[3],feature_alpha_0[3],feature_beta_0[3])
    ratio1 = ratio_of_values(feature_delta_1[3],feature_theta_1[3],feature_alpha_1[3],feature_beta_1[3])
    ratio2 = ratio_of_values(feature_delta_2[3],feature_theta_2[3],feature_alpha_2[3],feature_beta_2[3])
    ratio3 = ratio_of_values(feature_delta_3[3],feature_theta_3[3],feature_alpha_3[3],feature_beta_3[3])
    ratio4 = ratio_of_values(feature_delta_4[3],feature_theta_4[3],feature_alpha_4[3],feature_beta_4[3])
    ratio5 = ratio_of_values(feature_delta_5[3],feature_theta_5[3],feature_alpha_5[3],feature_beta_5[3])
    
    zip_delta = zip_feature(feature_delta_0,feature_delta_1,feature_delta_2,
                        feature_delta_3,feature_delta_4,feature_delta_5)
    zip_theta = zip_feature(feature_theta_0,feature_theta_1,feature_theta_2,
                        feature_theta_3,feature_theta_4,feature_theta_5)
    zip_alpha = zip_feature(feature_alpha_0,feature_alpha_1,feature_alpha_2,
                        feature_alpha_3,feature_alpha_4,feature_alpha_5)
    zip_beta = zip_feature(feature_beta_0,feature_beta_1,feature_beta_2,
                       feature_beta_3,feature_beta_4,feature_beta_5)
    zip_ratio = zip_feature(ratio0,ratio1,ratio2,ratio3,ratio4,ratio5)
    X = zip_delta,zip_theta,zip_alpha,zip_beta
    X = np.array(X)
    X = X.reshape([-1,len(sleep_stage)])
    Y = sleep_stage
    open('weka_subject_'+index[index_num]+'.arff', 'w').close() # clear the file
    with open('weka_subject_'+index[index_num]+'.arff', 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',quotechar=' ', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['@relation subject'+index[index_num]+'\n'])
        
        for i in range(0,6):
            temp_i = str(i)
            csvwriter.writerow(['@attribute delta_abs_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute delta_angle_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute delta_mqv'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute delta_abs_min'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute delta_abs_max'+temp_i+' numeric'])
        for i in range(0,6):
            temp_i = str(i)     
            csvwriter.writerow(['@attribute theta_abs_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute theta_angle_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute theta_mqv'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute theta_abs_min'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute theta_abs_max'+temp_i+' numeric'])
        for i in range(0,6):
            temp_i = str(i)        
            csvwriter.writerow(['@attribute alpha_abs_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute alpha_angle_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute alpha_mqv'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute alpha_abs_min'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute alpha_abs_max'+temp_i+' numeric'])
        for i in range(0,6):
            temp_i = str(i)
            csvwriter.writerow(['@attribute beta_abs_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute beta_angle_mean'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute beta_mqv'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute beta_abs_min'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute beta_abs_max'+temp_i+' numeric'])
        for i in range(0,6):
            temp_i = str(i)
            csvwriter.writerow(['@attribute ratio_delta'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute ratio_theta'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute ratio_alpha'+temp_i+' numeric'])
            csvwriter.writerow(['@attribute ratio_beta'+temp_i+' numeric'])
                
        csvwriter.writerow(['@attribute sleep_stage {-1,0,1,2,3,4,5}'])
        csvwriter.writerow(['\n@data'])
        for j in range(0,len(Y)):
            temp_writer = []
            for i in range(0,len(X)):
                temp_writer.append(X[i][j])
            for i in range(0,len(zip_ratio)):
                temp_writer.append(zip_ratio[i][j])
            temp_writer.append(Y[j])
            csvwriter.writerow(temp_writer)